In [1]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import numpy as np
import pandas as pd

import touchscreen_toolbox as tb

FOLDER = 'tests'
DLC_CFG = 'touchscreen_toolbox/DLC/config.yaml'

In [8]:
for path, sub, files in list(os.walk(FOLDER)):
    
    if files:
        videos = [f for f in files if f.endswith('.mp4')]  # all videos
        videos = [f for f in videos if not tb.preprocess.is_preprocess(f)]  # all valid videos
        for v in videos:
            v = os.path.join(path, v)
            print(v)
            tb.preprocess.preprocess(v)
            print('')
            
    

tests\20220101\2_clip.mp4

tests\20220101\6_clip.mp4

tests\20220101\7_clip.mp4
Increasing brightness for 'tests\20220101\7_clip.mp4'
Preprocessed video 'tests\20220101\7_clip_bright.mp4' exists, removing it

tests\20220102\4_clip.mp4

